In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
# import cv2
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
# from tensorflow.keras.layers.normalization import BatchNormalization
import os
import seaborn as sns
# from keras.applications import DenseNet121
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

In [2]:
train_data_path = 'Training'
test_data_path = 'Testing'

In [3]:
x_train = []
y_train = []
x_test = []
y_test = []

In [4]:
# Function to process images in a folder
def process_images(folder_path, images_list, labels_list):
    for class_folder in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_folder)
        if not os.path.isdir(class_path):
            continue
        
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            image = Image.open(image_path)
            
            # Convert the image to grayscale
            image = image.convert('L')
            
            # Resize the image to 64x64
            image = image.resize((64, 64))
            
            # Convert the image to a NumPy array
            image_array = np.array(image)
            
            # Normalize the image data (if needed)
            image_array = image_array / 255.0
            
            # Append the processed image and its label to the lists
            images_list.append(image_array)
            labels_list.append(class_folder)


In [5]:
# Process training images
process_images(train_data_path, x_train, y_train)

# Process testing images
process_images(test_data_path, x_test, y_test)

In [6]:
x_tr=[]
x_ts=[]
y_tr=[]
y_ts=[]

In [7]:
x_tr = np.array(x_train)
y_tr = np.array(y_train)

In [8]:
x_ts = np.array(x_test)
y_ts = np.array(y_test)

In [9]:
# # Convert the lists to NumPy arrays
# x_train = np.array(x_train)
# x_test = np.array(x_test)

In [10]:
x_tr.shape,y_tr.shape,x_ts.shape,y_ts.shape

((84096, 64, 64), (84096,), (21024, 64, 64), (21024,))

In [11]:
# Convert class labels to categorical format using one-hot encoding
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_tr)
y_test_encoded = label_encoder.transform(y_ts)

onehot_encoder = OneHotEncoder(sparse=False)
y_tr1 = onehot_encoder.fit_transform(y_train_encoded.reshape(-1, 1))
y_ts1 = onehot_encoder.transform(y_test_encoded.reshape(-1, 1))

In [12]:
x_tr.shape,y_tr1.shape,x_ts.shape,y_ts1.shape

((84096, 64, 64), (84096, 5), (21024, 64, 64), (21024, 5))

In [13]:
# Reshape the data by adding an extra dimension
x_tr = np.expand_dims(x_tr, axis=-1)
x_ts = np.expand_dims(x_ts, axis=-1)

In [14]:
x_tr.shape,y_tr1.shape,x_ts.shape,y_ts1.shape

((84096, 64, 64, 1), (84096, 5), (21024, 64, 64, 1), (21024, 5))

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [16]:
# Define the shape of the input data
input_shape = (64, 64, 1)

# Define the number of classes
num_classes = 5

# Define the 2D CNN model architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(6, 6), activation='relu', padding='same', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(4, 4), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
# Fully connected
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
# Output layer
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        1184      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 21, 21, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 21, 21, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        32832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 64)        0

In [17]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Fit the model to the training data
history=model.fit(x_tr, y_tr1, batch_size=64, epochs=50)

Epoch 1/50
1314/1314 [==============================] - 143s 108ms/step - loss: 0.8754 - accuracy: 0.6795
Epoch 2/50
1314/1314 [==============================] - 144s 110ms/step - loss: 0.4625 - accuracy: 0.8273
Epoch 3/50
1314/1314 [==============================] - 146s 111ms/step - loss: 0.3564 - accuracy: 0.8673
Epoch 4/50
1314/1314 [==============================] - 143s 109ms/step - loss: 0.3053 - accuracy: 0.8845
Epoch 5/50
1314/1314 [==============================] - 141s 107ms/step - loss: 0.2746 - accuracy: 0.8951
Epoch 6/50
1314/1314 [==============================] - 141s 107ms/step - loss: 0.2499 - accuracy: 0.9032
Epoch 7/50
1314/1314 [==============================] - 140s 107ms/step - loss: 0.2345 - accuracy: 0.9098
Epoch 8/50
1314/1314 [==============================] - 141s 107ms/step - loss: 0.2175 - accuracy: 0.9146
Epoch 9/50
1314/1314 [==============================] - 142s 108ms/step - loss: 0.2053 - accuracy: 0.9208
Epoch 10/50
1314/1314 [=======================

In [ ]:
# _, accuracy = model.evaluate(x_ts, y_ts1, batch_size=64, verbose=0)

In [ ]:
# accuracy

In [ ]:
prediction = model.predict(x_ts)

In [ ]:
prediction_NN=np.argmax(prediction, axis=-1)

In [ ]:
prediction_NN

In [ ]:
y_ts1_1=np.argmax(y_ts1, axis=-1)

In [ ]:
y_ts1_1

In [ ]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_ts1_1, prediction_NN))

In [ ]:
#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_ts1_1, prediction_NN)
print(cm)

In [ ]:
from sklearn.metrics import classification_report

# print(class_labels)
target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4']
print(classification_report(y_ts1_1, prediction_NN, target_names=target_names,digits=3))

In [ ]:
model.save('M2_copy_all_imgs_2lyr.h5')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
# class_dict = {0: 'Class 0',
#               1: 'Class 1',
#               2: 'Class 2',
#               3: 'Class 3',
#               4: 'Class 4'}

# multiclass = np.array([[11825,     7,     3,    71,    20],
#                        [149,  1293,     0,    19,   116],
#                        [19,     0,  1509,     0,     8],
#                        [178,     0,     0,   973,   110],
#                        [9,     8,     0,    56,  4651]])

# fig, ax = plot_confusion_matrix(
#     conf_mat=multiclass,
#     class_names=class_dict.values(),show_normed=True,colorbar=True)